# `adf_summary` Demo
This file demostrates how to extract data from the "PD_DATA_2017.pkl", create a summary table, and perform some tests. See the documentation for details on the applied functions: //dcd02cb-evs02/ValidationTeam1/Projects/CRV_PYLIB/docs/html

In [1]:
# imports and options
import pandas as pd
from crv.validation.calibration import (jeffreys_test, binomial_test_norm_approx,
    binomial_test, confidence_intervals, hosmer_lemeshow_test)
from crv.validation.differentiation import (auc, herfindahl_index)
from crv.validation.summary import adf_summary

from IPython.display import display

Invoking __init__.py for crv.utils


### Extract data for use in the notebook
The data is extracted from "PD_DATA_2017.pkl", which is stored in the sharepoint environment "Nordea\CRV_PYLIB - Documents\Testing Notebooks\test_data". 

Note: please fill in your user_id in the cell below.

In [2]:
# extract data
df_sco = pd.DataFrame()
infile = "PD_DATA_2017.pkl"
user_id = "G85544"
path = "C:\\Users\\" + user_id + "\\Nordea\\CRV_PYLIB - Documents\\Testing Notebooks\\test_data\\"

pd_data = pd.read_pickle(path + infile)

In [3]:
# look at the columns in the returned data
print("Columns found in the PD FIRB dataset:")
display(pd_data.columns)

Columns found in the PD FIRB dataset:


Index(['CALMONTH', 'CALDAY', 'FGRC', 'BA_1EXPCLO', 'kundnrin', 'group',
       'BA_1RWAAR', 'country', 'FBP13CTRY', 'FGLFLG', 'FCRU',
       'naceindustrycategory', 'FGRAD_MET', 'BA_1PD', 'FRATINGO', 'FSMEEFLG',
       '_TYPE_', '_FREQ_', 'EAD', 'ORGEXP', 'LOCAL_SYSTEM_ID',
       'kundnrin_original', 'bp1_partner_local_mother', 'BP35', 'group_name',
       'ratmodel', 'system', 'database', 'ratingnum', 'group_replaced',
       'rating', 'default', 'PD'],
      dtype='object')

### How to use the `adf_summary` function
Now that we have the data, we can use it to perform some tests. The `adf_summary` function creates a summary table according to the given summary and grouping columns. Therefore, we can put the dataframe we just retrieved ('pd_data') into the function, which will return a summary table with the actual default frequency (ADF), hence the name of the function.

The cells below demonstrate the different tests that can be run on the summary table, and further information can be found in the CRV_PYLIB documentation, or by running `help(adf_summary)` in a Jupyter cell.

In [4]:
# run help(adf_summary) to obtain information on the function
help(adf_summary)

Help on function adf_summary in module crv.validation.summary:

adf_summary(data, customer_col, default_col, group_list=None, sum_list=None)
    A summary table class, from which tests can be called.
    
    Given the data set ready for summarizing (e.g. defaults flagged,
    duplicates removed), the constructor creates a summary table
    according to the given summary and grouping columns. The
    constructor for this table requires that the following data is
    present in the input table: unique customer id, default
    flag.
    
    Args:
        data (pandas.DataFrame): The data.
    
        customer_col (string): Column name of the unique identifier (e.g.
         customer, group).
    
        default_col (string): Column name with default flag. Methods assume
         that the default column is an integer with 1 indicating default and 0
         indicating nondefault.
    
        group_list (list): List of column names that will be used for
         grouping in the ADF tab

In [5]:
# create the AdfSummary object...
adf_df = adf_summary(data=pd_data,
                     default_col='default',
                     customer_col='group',
                     sum_list=['ORGEXP', 'EAD', 'BA_1RWAAR'],
                     group_list='rating')
# and look at the summary table, with ADF computed
display(adf_df)

rating     N   D       ADF        ORGEXP           EAD     BA_1RWAAR
0      6+  1758   0  0.000000  3.655340e+09  3.525340e+09  3.898652e+08
1       6  1323   0  0.000000  3.062111e+09  2.930436e+09  3.124934e+08
2      6-  1488   0  0.000000  2.879478e+09  2.548227e+09  4.365698e+08
3      5+  1820   0  0.000000  3.021112e+09  2.629413e+09  4.287523e+08
4       5  2832   3  0.001059  4.277538e+09  3.724384e+09  9.558955e+08
5      5-  3474   2  0.000576  5.503468e+09  4.923292e+09  1.508490e+09
6      4+  3546   5  0.001410  5.744959e+09  4.747531e+09  1.741255e+09
7       4  3776   6  0.001589  5.262446e+09  4.608910e+09  2.015266e+09
8      4-  3465  13  0.003752  3.445783e+09  3.191514e+09  1.540314e+09
9      3+  2312  10  0.004325  1.973293e+09  1.818638e+09  1.088159e+09
10      3  1694  29  0.017119  8.731675e+08  7.561898e+08  4.941023e+08
11     3-  1532  27  0.017624  8.408553e+08  7.817895e+08  6.303225e+08
12     2+   662  44  0.066465  5.221445e+08  5.115818e+08  5.471096e+08
13      2   409  27  0.066015  1.787199e+08  1.546579e+08  1.830378e+08
14     2-   328  21  0.064024  1.823506e+08  1.308786e+08  1.398609e+08
15     1+   186  16  0.086022  1.059339e+08  8.197067e+07  1.335000e+08
16      1   151  19  0.125828  3.103435e+07  2.445644e+07  2.687417e+07
17     1-   138  26  0.188406  3.132664e+07  3.006551e+07  3.803484e+07

In [6]:
# Add the PD predictions per rating class
pd_data = pd_data.groupby('rating')['PD'].mean().reset_index()
adf_df = adf_df.merge(pd_data, left_on='rating', right_on='rating')
# and look at the summary table, with a PD column
display(adf_df)

rating     N   D       ADF        ORGEXP           EAD     BA_1RWAAR  \
0      6+  1758   0  0.000000  3.655340e+09  3.525340e+09  3.898652e+08   
1       6  1323   0  0.000000  3.062111e+09  2.930436e+09  3.124934e+08   
2      6-  1488   0  0.000000  2.879478e+09  2.548227e+09  4.365698e+08   
3      5+  1820   0  0.000000  3.021112e+09  2.629413e+09  4.287523e+08   
4       5  2832   3  0.001059  4.277538e+09  3.724384e+09  9.558955e+08   
5      5-  3474   2  0.000576  5.503468e+09  4.923292e+09  1.508490e+09   
6      4+  3546   5  0.001410  5.744959e+09  4.747531e+09  1.741255e+09   
7       4  3776   6  0.001589  5.262446e+09  4.608910e+09  2.015266e+09   
8      4-  3465  13  0.003752  3.445783e+09  3.191514e+09  1.540314e+09   
9      3+  2312  10  0.004325  1.973293e+09  1.818638e+09  1.088159e+09   
10      3  1694  29  0.017119  8.731675e+08  7.561898e+08  4.941023e+08   
11     3-  1532  27  0.017624  8.408553e+08  7.817895e+08  6.303225e+08   
12     2+   662  44  0.066465  5.221445e+08  5.115818e+08  5.471096e+08   
13      2   409  27  0.066015  1.787199e+08  1.546579e+08  1.830378e+08   
14     2-   328  21  0.064024  1.823506e+08  1.308786e+08  1.398609e+08   
15     1+   186  16  0.086022  1.059339e+08  8.197067e+07  1.335000e+08   
16      1   151  19  0.125828  3.103435e+07  2.445644e+07  2.687417e+07   
17     1-   138  26  0.188406  3.132664e+07  3.006551e+07  3.803484e+07   

          PD  
0   0.000300  
1   0.000340  
2   0.000413  
3   0.000652  
4   0.001019  
5   0.001505  
6   0.002596  
7   0.003381  
8   0.005724  
9   0.008793  
10  0.013940  
11  0.024615  
12  0.073184  
13  0.084025  
14  0.097121  
15  0.147371  
16  0.188898  
17  0.247010

In [7]:
# jeffreys test, appears as a 'p-value' column
print("Demo: Results of Jeffreys test")
j_test = jeffreys_test(adf_df, "D", "N", "PD", alpha=0.95)
display(j_test)

Demo: Results of Jeffreys test


rating     N   D       ADF        ORGEXP           EAD     BA_1RWAAR  \
0      6+  1758   0  0.000000  3.655340e+09  3.525340e+09  3.898652e+08   
1       6  1323   0  0.000000  3.062111e+09  2.930436e+09  3.124934e+08   
2      6-  1488   0  0.000000  2.879478e+09  2.548227e+09  4.365698e+08   
3      5+  1820   0  0.000000  3.021112e+09  2.629413e+09  4.287523e+08   
4       5  2832   3  0.001059  4.277538e+09  3.724384e+09  9.558955e+08   
5      5-  3474   2  0.000576  5.503468e+09  4.923292e+09  1.508490e+09   
6      4+  3546   5  0.001410  5.744959e+09  4.747531e+09  1.741255e+09   
7       4  3776   6  0.001589  5.262446e+09  4.608910e+09  2.015266e+09   
8      4-  3465  13  0.003752  3.445783e+09  3.191514e+09  1.540314e+09   
9      3+  2312  10  0.004325  1.973293e+09  1.818638e+09  1.088159e+09   
10      3  1694  29  0.017119  8.731675e+08  7.561898e+08  4.941023e+08   
11     3-  1532  27  0.017624  8.408553e+08  7.817895e+08  6.303225e+08   
12     2+   662  44  0.066465  5.221445e+08  5.115818e+08  5.471096e+08   
13      2   409  27  0.066015  1.787199e+08  1.546579e+08  1.830378e+08   
14     2-   328  21  0.064024  1.823506e+08  1.308786e+08  1.398609e+08   
15     1+   186  16  0.086022  1.059339e+08  8.197067e+07  1.335000e+08   
16      1   151  19  0.125828  3.103435e+07  2.445644e+07  2.687417e+07   
17     1-   138  26  0.188406  3.132664e+07  3.006551e+07  3.803484e+07   

          PD   p-value  Passed  
0   0.000300  0.695666    True  
1   0.000340  0.657208    True  
2   0.000413  0.732505    True  
3   0.000652  0.876659    True  
4   0.001019  0.433391    True  
5   0.001505  0.936854    True  
6   0.002596  0.927731    True  
7   0.003381  0.980522    True  
8   0.005724  0.945468    True  
9   0.008793  0.993988    True  
10  0.013940  0.133431    True  
11  0.024615  0.966469    True  
12  0.073184  0.742738    True  
13  0.084025  0.908891    True  
14  0.097121  0.982923    True  
15  0.147371  0.993819    True  
16  0.188898  0.980148    True  
17  0.247010  0.948076    True

In [8]:
# binomial test using a normal approximation, in the 'Lower Confidence Limit' and 'Upper Confidence Limit' columns
bt_norm = binomial_test_norm_approx(adf_df, "D", "N", "PD", alpha=0.95)
print('Demo: Results of binomial test using normal approximation')
display(bt_norm)

Demo: Results of binomial test using normal approximation


rating     N   D       ADF        ORGEXP           EAD     BA_1RWAAR  \
0      6+  1758   0  0.000000  3.655340e+09  3.525340e+09  3.898652e+08   
1       6  1323   0  0.000000  3.062111e+09  2.930436e+09  3.124934e+08   
2      6-  1488   0  0.000000  2.879478e+09  2.548227e+09  4.365698e+08   
3      5+  1820   0  0.000000  3.021112e+09  2.629413e+09  4.287523e+08   
4       5  2832   3  0.001059  4.277538e+09  3.724384e+09  9.558955e+08   
5      5-  3474   2  0.000576  5.503468e+09  4.923292e+09  1.508490e+09   
6      4+  3546   5  0.001410  5.744959e+09  4.747531e+09  1.741255e+09   
7       4  3776   6  0.001589  5.262446e+09  4.608910e+09  2.015266e+09   
8      4-  3465  13  0.003752  3.445783e+09  3.191514e+09  1.540314e+09   
9      3+  2312  10  0.004325  1.973293e+09  1.818638e+09  1.088159e+09   
10      3  1694  29  0.017119  8.731675e+08  7.561898e+08  4.941023e+08   
11     3-  1532  27  0.017624  8.408553e+08  7.817895e+08  6.303225e+08   
12     2+   662  44  0.066465  5.221445e+08  5.115818e+08  5.471096e+08   
13      2   409  27  0.066015  1.787199e+08  1.546579e+08  1.830378e+08   
14     2-   328  21  0.064024  1.823506e+08  1.308786e+08  1.398609e+08   
15     1+   186  16  0.086022  1.059339e+08  8.197067e+07  1.335000e+08   
16      1   151  19  0.125828  3.103435e+07  2.445644e+07  2.687417e+07   
17     1-   138  26  0.188406  3.132664e+07  3.006551e+07  3.803484e+07   

          PD        k_L        k_H       LCL       UCL  Passed  
0   0.000300  -0.895758   1.950558 -0.000510  0.001110    True  
1   0.000340  -0.864477   1.764117 -0.000653  0.001333    True  
2   0.000413  -0.921610   2.150698 -0.000619  0.001445    True  
3   0.000652  -0.947712   3.320992 -0.000521  0.001825    True  
4   0.001019  -0.442017   6.213633 -0.000156  0.002194    True  
5   0.001505   0.750163   9.706577  0.000216  0.002794    True  
6   0.002596   3.266525  15.144307  0.000921  0.004271    True  
7   0.003381   5.775464  19.757848  0.001530  0.005232    True  
8   0.005724  11.129978  28.537342  0.003212  0.008236    True  
9   0.008793  11.531238  29.127594  0.004988  0.012598   False  
10  0.013940  14.156610  33.072110  0.008357  0.019523    True  
11  0.024615  25.823367  49.596993  0.016856  0.032374    True  
12  0.073184  35.314264  61.581352  0.053345  0.093023    True  
13  0.084025  23.369691  45.362759  0.057139  0.110911    True  
14  0.097121  21.344390  42.366986  0.065074  0.129168   False  
15  0.147371  17.935757  36.886255  0.096429  0.198313   False  
16  0.188898  19.096284  37.950912  0.126465  0.251331   False  
17  0.247010  24.157603  44.017157  0.175055  0.318965    True

In [9]:
# binomial test (no approximation), in the 'Lower Confidence Limit' and 'Upper Confidence Limit' columns
binom_test = binomial_test(adf_df, "D", "N", "PD", alpha=0.95)
print('Demo: Results of binomial test')
display(binom_test)

Demo: Results of binomial test


rating     N   D       ADF        ORGEXP           EAD     BA_1RWAAR  \
0      6+  1758   0  0.000000  3.655340e+09  3.525340e+09  3.898652e+08   
1       6  1323   0  0.000000  3.062111e+09  2.930436e+09  3.124934e+08   
2      6-  1488   0  0.000000  2.879478e+09  2.548227e+09  4.365698e+08   
3      5+  1820   0  0.000000  3.021112e+09  2.629413e+09  4.287523e+08   
4       5  2832   3  0.001059  4.277538e+09  3.724384e+09  9.558955e+08   
5      5-  3474   2  0.000576  5.503468e+09  4.923292e+09  1.508490e+09   
6      4+  3546   5  0.001410  5.744959e+09  4.747531e+09  1.741255e+09   
7       4  3776   6  0.001589  5.262446e+09  4.608910e+09  2.015266e+09   
8      4-  3465  13  0.003752  3.445783e+09  3.191514e+09  1.540314e+09   
9      3+  2312  10  0.004325  1.973293e+09  1.818638e+09  1.088159e+09   
10      3  1694  29  0.017119  8.731675e+08  7.561898e+08  4.941023e+08   
11     3-  1532  27  0.017624  8.408553e+08  7.817895e+08  6.303225e+08   
12     2+   662  44  0.066465  5.221445e+08  5.115818e+08  5.471096e+08   
13      2   409  27  0.066015  1.787199e+08  1.546579e+08  1.830378e+08   
14     2-   328  21  0.064024  1.823506e+08  1.308786e+08  1.398609e+08   
15     1+   186  16  0.086022  1.059339e+08  8.197067e+07  1.335000e+08   
16      1   151  19  0.125828  3.103435e+07  2.445644e+07  2.687417e+07   
17     1-   138  26  0.188406  3.132664e+07  3.006551e+07  3.803484e+07   

          PD  k_L  k_H       LCL       UCL  Passed  
0   0.000300    0    3  0.000000  0.002096    True  
1   0.000340    0    3  0.000000  0.002784    True  
2   0.000413    0    3  0.000000  0.002476    True  
3   0.000652    1    5  0.000014  0.003058   False  
4   0.001019    1    7  0.000086  0.002549    True  
5   0.001505    2   11  0.000467  0.003356    True  
6   0.002596    5   17  0.001161  0.004813    True  
7   0.003381    7   20  0.001643  0.005545   False  
8   0.005724   12   29  0.003305  0.008550    True  
9   0.008793   13   30  0.005292  0.013329   False  
10  0.013940   15   34  0.008626  0.020303    True  
11  0.024615   27   50  0.017061  0.033137    True  
12  0.073184   36   62  0.053942  0.094988    True  
13  0.084025   24   46  0.058255  0.114229    True  
14  0.097121   22   43  0.065122  0.131467   False  
15  0.147371   19   37  0.097893  0.204132   False  
16  0.188898   20   38  0.126896  0.256725   False  
17  0.247010   25   45  0.177052  0.326910    True

In [10]:
# AUROC score
auc = auc(adf_df, "D", "N", "PD")
print(f'AUC score: {auc}')

AUC score: 0.9034712602602455


In [11]:
# Herfindahl index
h_test = herfindahl_index(adf_df, "PD", weight=None, CV_init=0.725)
print('Demo: Results of Herfindahl index test')
print(f'HI: {h_test[0]}, CV: {h_test[1]}, p-value: {h_test[2]}')

Demo: Results of Herfindahl index test
HI: 0.14507547052318082, CV: 0.7217548617475079, p-value: 0.5073190657312148


In [12]:
# confindence intervals, in the 'LCL' and 'UCL' columns
c_intervals = confidence_intervals(adf_df, "PD", rho=0.02, alpha=0.95)
print('Demo: Confidence intervals')
display(c_intervals)

Demo: Confidence intervals


rating     N   D       ADF        ORGEXP           EAD     BA_1RWAAR  \
0      6+  1758   0  0.000000  3.655340e+09  3.525340e+09  3.898652e+08   
1       6  1323   0  0.000000  3.062111e+09  2.930436e+09  3.124934e+08   
2      6-  1488   0  0.000000  2.879478e+09  2.548227e+09  4.365698e+08   
3      5+  1820   0  0.000000  3.021112e+09  2.629413e+09  4.287523e+08   
4       5  2832   3  0.001059  4.277538e+09  3.724384e+09  9.558955e+08   
5      5-  3474   2  0.000576  5.503468e+09  4.923292e+09  1.508490e+09   
6      4+  3546   5  0.001410  5.744959e+09  4.747531e+09  1.741255e+09   
7       4  3776   6  0.001589  5.262446e+09  4.608910e+09  2.015266e+09   
8      4-  3465  13  0.003752  3.445783e+09  3.191514e+09  1.540314e+09   
9      3+  2312  10  0.004325  1.973293e+09  1.818638e+09  1.088159e+09   
10      3  1694  29  0.017119  8.731675e+08  7.561898e+08  4.941023e+08   
11     3-  1532  27  0.017624  8.408553e+08  7.817895e+08  6.303225e+08   
12     2+   662  44  0.066465  5.221445e+08  5.115818e+08  5.471096e+08   
13      2   409  27  0.066015  1.787199e+08  1.546579e+08  1.830378e+08   
14     2-   328  21  0.064024  1.823506e+08  1.308786e+08  1.398609e+08   
15     1+   186  16  0.086022  1.059339e+08  8.197067e+07  1.335000e+08   
16      1   151  19  0.125828  3.103435e+07  2.445644e+07  2.687417e+07   
17     1-   138  26  0.188406  3.132664e+07  3.006551e+07  3.803484e+07   

          PD       LCL       UCL  Passed  
0   0.000300  0.000107  0.000616   False  
1   0.000340  0.000123  0.000694   False  
2   0.000413  0.000151  0.000836   False  
3   0.000652  0.000248  0.001294   False  
4   0.001019  0.000403  0.001982    True  
5   0.001505  0.000615  0.002874   False  
6   0.002596  0.001113  0.004823    True  
7   0.003381  0.001485  0.006195    True  
8   0.005724  0.002641  0.010186    True  
9   0.008793  0.004228  0.015257    True  
10  0.013940  0.007019  0.023499    True  
11  0.024615  0.013157  0.039923    True  
12  0.073184  0.044358  0.108928    True  
13  0.084025  0.051819  0.123531    True  
14  0.097121  0.061017  0.140889    True  
15  0.147371  0.097938  0.205131   False  
16  0.188898  0.130104  0.255931   False  
17  0.247010  0.177262  0.324233    True

In [13]:
# Hosmer Lemeshow test
adf_df, x_sq, dof, pvalue = hosmer_lemeshow_test(adf_df, 'rating', 'D', 'N', 'PD')
print("Demo: Hosmer-Lemeshow test")
print(f"P-value: {pvalue}. Chi-square test statistic: {x_sq}. Degrees of freedom: {dof}.")
display(adf_df)

Demo: Hosmer-Lemeshow test
P-value: 0.0006285882944104686. Chi-square test statistic: 40.634393313290715. Degrees of freedom: 16.


rating     N        PD         ED   D      X_sq  relative_X_sq
0      6+  1758  0.000300   0.527400   0  0.527558       0.012983
1       6  1323  0.000340   0.449820   0  0.449973       0.011074
2      6-  1488  0.000413   0.614544   0  0.614798       0.015130
3      5+  1820  0.000652   1.186640   0  1.187414       0.029222
4       5  2832  0.001019   2.885808   3  0.004523       0.000111
5      5-  3474  0.001505   5.228370   2  1.996431       0.049132
6      4+  3546  0.002596   9.205416   5  1.926209       0.047403
7       4  3776  0.003381  12.766656   6  3.598669       0.088562
8      4-  3465  0.005724  19.833660  13  2.368083       0.058278
9      3+  2312  0.008793  20.329416  10  5.294955       0.130307
10      3  1694  0.013940  23.614360  29  1.245647       0.030655
11     3-  1532  0.024615  37.710180  27  3.118594       0.076748
12     2+   662  0.073184  48.447808  44  0.440580       0.010843
13      2   409  0.084025  34.366225  27  1.723751       0.042421
14     2-   328  0.097121  31.855688  21  4.097304       0.100833
15     1+   186  0.147371  27.411006  16  5.571380       0.137110
16      1   151  0.188898  28.523598  19  3.920327       0.096478
17     1-   138  0.247010  34.087380  26  2.548196       0.062710